In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2 as cv
from numpy.random import seed
seed(42)
import pickle

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob 
%matplotlib inline

Some learning parameters

In [ ]:
IMG_CHANNELS = 3
#TRAIN_SIZE=80000
TRAIN_SIZE = 8000
BATCH_SIZE = 64
EPOCHS = 30


Data exploration

In [3]:
#I used cropped images
train_labels = pd.read_csv('train_labels.csv')
train_labels['label'].value_counts()


0    130908
1     89117
Name: label, dtype: int64

In [4]:
(train_labels.label.value_counts() / len(train_labels)).to_frame()

,label
0,0.594969
1,0.405031


Balancing the data

In [5]:
train_neg = train_labels[train_labels['label']==0].sample(TRAIN_SIZE,random_state=45)
train_pos = train_labels[train_labels['label']==1].sample(TRAIN_SIZE,random_state=45)

train_data = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

train_data = shuffle(train_data)


In [6]:
train_data['label'].value_counts()

1    8000
0    8000
Name: label, dtype: int64

In [7]:
def append_ext(fn):
    return fn+".tif"

In [8]:
y = train_data['label']
train_df, valid_df = train_test_split(train_data, test_size=0.2, random_state=45, stratify=y)

print(train_df.shape)
print(valid_df.shape)

(12800, 2)
(3200, 2)


In [9]:
train_df['id'] = train_df['id'].apply(append_ext)
valid_df['id'] = valid_df['id'].apply(append_ext)
train_df.head()

,id,label
12571,4e2c9a6d0be94031e10013cffc9dd8747520e9bd.tif,1
875,f87538389122888cd4c56cbe1b9b22b79a2a1c93.tif,0
4506,43c463fdc93bc0ad76bb64b5a5b3761648b70c2d.tif,0
12453,816336cfb3623fa6693bf0a0012b897dddec6b7f.tif,1
2101,72da772f3577ecc920101422201d0d3f03e87c07.tif,0


Image generator that load images in batches

In [10]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [11]:
BATCH_SIZE = 64
train_path = './train'
train_df['label'] = train_df['label'].astype(str)
valid_df['label'] = valid_df['label'].astype(str)

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)

valid_loader = valid_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)


Found 12800 validated image filenames belonging to 2 classes.
Found 3200 validated image filenames belonging to 2 classes.


In [12]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

200
50


In [45]:
cnn = Sequential([
    Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape=(32,32,3)),
    #Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    #Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.4),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    #Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    #Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.3),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    #Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    #Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.2),
    BatchNormalization(),

    Flatten(),
     Dense(128, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    BatchNormalization(),
    Dense(2, activation='softmax')
])


Round one training

In [46]:
opt = tf.keras.optimizers.Adam(0.001)
cnn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [47]:
h1 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
200/200 [==============================] - ETA: 0s - loss: 0.5679 - accuracy: 0.7153 - auc_5: 0.7869WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x17c4e5e50> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AU

In [48]:
tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)

Round two training

In [49]:
h2 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

Epoch 1/30
200/200 [==============================] - 8s 40ms/step - loss: 0.1922 - accuracy: 0.9251 - auc_5: 0.9755 - val_loss: 0.6874 - val_accuracy: 0.7647 - val_auc_5: 0.8260
Epoch 2/30
200/200 [==============================] - 7s 37ms/step - loss: 0.1849 - accuracy: 0.9280 - auc_5: 0.9772 - val_loss: 0.6927 - val_accuracy: 0.7675 - val_auc_5: 0.8286
Epoch 3/30
200/200 [==============================] - 7s 36ms/step - loss: 0.1893 - accuracy: 0.9238 - auc_5: 0.9757 - val_loss: 0.6903 - val_accuracy: 0.7656 - val_auc_5: 0.8263
Epoch 4/30
200/200 [==============================] - 7s 36ms/step - loss: 0.1839 - accuracy: 0.9288 - auc_5: 0.9763 - val_loss: 0.6796 - val_accuracy: 0.7691 - val_auc_5: 0.8289
Epoch 5/30
200/200 [==============================] - 7s 33ms/step - loss: 0.1779 - accuracy: 0.9323 - auc_5: 0.9780 - val_loss: 0.6847 - val_accuracy: 0.7738 - val_auc_5: 0.8312
Epoch 6/30
200/200 [==============================] - 6s 32ms/step - loss: 0.1665 - accuracy: 0.9375 - au

In [50]:
tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.00001)

Round three training

In [51]:
 h3 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

Epoch 1/30
200/200 [==============================] - 6s 32ms/step - loss: 0.1301 - accuracy: 0.9516 - auc_5: 0.9858 - val_loss: 0.7529 - val_accuracy: 0.7759 - val_auc_5: 0.8270
Epoch 2/30
200/200 [==============================] - 6s 31ms/step - loss: 0.1290 - accuracy: 0.9508 - auc_5: 0.9861 - val_loss: 0.7524 - val_accuracy: 0.7769 - val_auc_5: 0.8281
Epoch 3/30
200/200 [==============================] - 6s 30ms/step - loss: 0.1290 - accuracy: 0.9511 - auc_5: 0.9863 - val_loss: 0.7556 - val_accuracy: 0.7759 - val_auc_5: 0.8269
Epoch 4/30
200/200 [==============================] - 6s 32ms/step - loss: 0.1315 - accuracy: 0.9500 - auc_5: 0.9860 - val_loss: 0.7478 - val_accuracy: 0.7753 - val_auc_5: 0.8275
Epoch 5/30
200/200 [==============================] - 6s 31ms/step - loss: 0.1261 - accuracy: 0.9535 - auc_5: 0.9868 - val_loss: 0.7530 - val_accuracy: 0.7766 - val_auc_5: 0.8273
Epoch 6/30
200/200 [==============================] - 6s 31ms/step - loss: 0.1276 - accuracy: 0.9506 - au

Final outcome 95.3%, historical highest 95.53%